In [23]:
pip install scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ------------------------------ --------- 8.4/11.1 MB 41.3 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 35.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   --------- ------------------------------ 9.4/41.0 MB 45.7 MB/s eta 0:00:01
   ------------------- -------------------- 19.7/41.0 MB 46.0 MB/s eta 0:00:01
   ----------------------------- ---------- 29.9/41.0 MB 46.6 MB/s eta 0:00:01
   ---------------------------------------  40.1/41.0 MB 46.6 MB/s eta 0:00:01
   ---------------------------------------- 41.0/41.0 MB 38.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
# Cargar el CSV
import pandas as pd
from deep_translator import GoogleTranslator

# Cargar el archivo original
df = pd.read_csv(r'C:\Users\Guillermo\Downloads\archive\text.csv')


#creamos un dataframe con los misma cantidad de ejemplos para cada emocion
label0 = df[df['label'] == 0].sample(n=1000, random_state=1)
label1 = df[df['label'] == 1].sample(n=1000, random_state=1)
label2 = df[df['label'] == 2].sample(n=1000, random_state=1)
label3 = df[df['label'] == 3].sample(n=1000, random_state=1)
label4 = df[df['label'] == 4].sample(n=1000, random_state=1)
label5 = df[df['label'] == 5].sample(n=1000, random_state=1)

df_balanced = pd.concat([label0, label1, label2, label3, label4, label5], ignore_index=True)

# Traducir toda la columna 'text' al español
df_balanced['text'] = df_balanced['text'].head(6000).apply(lambda x: GoogleTranslator(source='auto', target='es').translate(str(x)))

In [72]:

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report

"""
sadness (0), joy (1), love (2), anger (3), fear (4), and surprise (5).
"""
#df_traducido = df.iloc[:3500,:]

X = df_balanced['text']
y = df_balanced['label']

vector = TfidfVectorizer()
X_vectoriced = vector.fit_transform(X)

X_train,X_test, y_train, y_test = train_test_split(X_vectoriced,y, test_size=0.2,random_state=34)


model_lr = LogisticRegression(max_iter=1000,class_weight='balanced')
model_lr.fit(X_train,y_train)
print(model_lr.score(X_test,y_test))
y_predict = model_lr.predict(X_test)

cm = confusion_matrix(y_test,y_predict)
cr = classification_report(y_test,y_predict)
print(cr)
print(cm)



0.725
              precision    recall  f1-score   support

           0       0.75      0.64      0.69       193
           1       0.61      0.69      0.65       203
           2       0.63      0.75      0.69       198
           3       0.78      0.76      0.77       206
           4       0.80      0.72      0.76       202
           5       0.83      0.79      0.81       198

    accuracy                           0.72      1200
   macro avg       0.73      0.72      0.73      1200
weighted avg       0.73      0.72      0.73      1200

[[123  23  21  14   7   5]
 [  9 140  31  10   9   4]
 [  7  29 149   6   4   3]
 [ 13  13  15 156   8   1]
 [  5  13  10   9 146  19]
 [  8  10  11   5   8 156]]


In [80]:
import joblib

# Guardar el modelo entrenado
joblib.dump(model_lr, 'modelo_logistico.pkl')

# Guardar el vectorizador
joblib.dump(vector, 'vectorizador_tfidf.pkl')


['vectorizador_tfidf.pkl']

In [73]:
print(df_balanced['label'].value_counts())


label
0    1000
1    1000
2    1000
3    1000
4    1000
5    1000
Name: count, dtype: int64


In [74]:
for i in range(20):
    if y_test.iloc[i] != y_predict[i]:
        print(f"Texto: {X.iloc[i]}")
        print(f"Verdadero: {y_test.iloc[i]}, Predicho: {y_predict[i]}")


Texto: No solo siento decepcionado por mi falta de fuerza de voluntad y, como si tomé varios pasos, en términos de mi entrenamiento de culturismo, sino que siento los antojos de toda su fuerza y ​​ahora tengo que usar X la fuerza de voluntad para negarles
Verdadero: 0, Predicho: 1
Texto: Tragué un larabar, así que no me sentiría privado si fuera el tiempo de bocadillo y todo lo que tenían era algo como lo que ves arriba
Verdadero: 4, Predicho: 1


In [60]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=300, criterion='gini',random_state=45)
model_rf.fit(X_train,y_train)
print(model_rf.score(X_test,y_test))
y_predict = model_rf.predict(X_test)

cm = confusion_matrix(y_test,y_predict)
cr = classification_report(y_test,y_predict)
print(cr)
print(cm)





0.6883333333333334
              precision    recall  f1-score   support

           0       0.60      0.63      0.62       193
           1       0.48      0.64      0.55       203
           2       0.72      0.69      0.70       198
           3       0.78      0.70      0.74       206
           4       0.80      0.67      0.73       202
           5       0.85      0.80      0.83       198

    accuracy                           0.69      1200
   macro avg       0.71      0.69      0.69      1200
weighted avg       0.71      0.69      0.69      1200

[[122  33  11  14  11   2]
 [ 25 129  27  11   7   4]
 [ 14  39 136   3   4   2]
 [ 20  26   7 144   8   1]
 [ 13  20   6   9 136  18]
 [  8  21   2   3   5 159]]


In [62]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train,y_train)
print(model.score(X_test,y_test))
y_predict = model.predict(X_test)

cm = confusion_matrix(y_test,y_predict)
cr = classification_report(y_test,y_predict)
print(cr)
print(cm)

0.7066666666666667
              precision    recall  f1-score   support

           0       0.76      0.60      0.67       193
           1       0.70      0.61      0.65       203
           2       0.64      0.77      0.70       198
           3       0.75      0.74      0.75       206
           4       0.71      0.72      0.71       202
           5       0.70      0.80      0.74       198

    accuracy                           0.71      1200
   macro avg       0.71      0.71      0.70      1200
weighted avg       0.71      0.71      0.71      1200

[[115  12  18  22  17   9]
 [  5 124  34  12  13  15]
 [  8  19 153   5   3  10]
 [ 11   9  12 153  13   8]
 [  5   6  11   8 145  27]
 [  8   6  10   3  13 158]]


In [67]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Parámetro de regularización
    'solver': ['liblinear', 'saga'],  # Solvers que podrías probar
    'max_iter': [8000, 10000],  # Máximo de iteraciones
}

grid_search = GridSearchCV(LogisticRegression(class_weight='balanced'), param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mejor modelo encontrado
print("Mejores parámetros:", grid_search.best_params_)

# Evaluación del mejor modelo
best_model = grid_search.best_estimator_
print(best_model.score(X_test, y_test))


Mejores parámetros: {'C': 10, 'max_iter': 8000, 'solver': 'liblinear'}
0.745


In [91]:
target=[]
for i in range(0,1000):
    dicc_ordenado = sorted(dicc[i], key=lambda x: x['score'],reverse=True)
    target.append(dicc_ordenado[0]['label'])

print(target)   
    
    #for item in dicc_ordenado:
        #print(item)
        #target.append(item[0]['label'])



['others', 'others', 'joy', 'others', 'others', 'joy', 'joy', 'joy', 'others', 'joy', 'anger', 'others', 'sadness', 'anger', 'anger', 'joy', 'others', 'others', 'anger', 'others', 'others', 'others', 'others', 'others', 'others', 'others', 'others', 'others', 'others', 'joy', 'others', 'joy', 'others', 'others', 'others', 'others', 'others', 'others', 'others', 'others', 'anger', 'others', 'others', 'joy', 'others', 'others', 'joy', 'others', 'others', 'joy', 'joy', 'joy', 'others', 'joy', 'anger', 'others', 'others', 'others', 'others', 'others', 'others', 'others', 'others', 'others', 'others', 'others', 'others', 'joy', 'others', 'others', 'others', 'others', 'others', 'joy', 'others', 'joy', 'others', 'others', 'others', 'others', 'disgust', 'sadness', 'others', 'others', 'others', 'others', 'others', 'others', 'others', 'others', 'others', 'others', 'sadness', 'fear', 'others', 'others', 'others', 'joy', 'joy', 'others', 'joy', 'others', 'joy', 'others', 'others', 'joy', 'joy', 'o